In [1]:
import pandas as pd
import numpy as np
from forex_python.converter import CurrencyRates
import requests
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import time
import re

In [17]:
#Importamos el archivo de excel
data_booking = pd.read_excel('Data Booking.xlsx')

#Visualizamos el dataframe y el tipo de datos que contiene
print(data_booking.dtypes)
data_booking

Detalle        object
Direccion      object
Dormitorios    object
Camas          object
Precio         object
Personas       object
Descripcion    object
dtype: object


,Detalle,Direccion,Dormitorios,Camas,Precio,Personas,Descripcion
0,"Hermoso departamento en el centro ciudad, casa...","420 Melgar, Arequipa, Perú",Apartamento de 2 dormitorios,1 cama individual,S/ 134,2 adultos · 0 niños · 1 habitación,"El Hermoso departamento en el centro ciudad, c..."
1,Departamento amplio y bonito en Arequipa 2do Piso,"Urbanización Colegio de ingenieros A9, Cerro C...",Apartamento de 1 dormitorio,1 cama doble,S/ 159,2 adultos · 0 niños · 1 habitación,El Departamento Amoblado en Urbanización Coleg...
2,Big apartment near historical center,"506 Lima, Arequipa, Perú",Apartamento,1 cama doble grande,S/ 179,2 adultos · 0 niños · 1 habitación,El Big apartment near historical center está s...
3,Furnished Aparments Arequipa,"Urbanización Campiña Dorada C - 1, Arequipa, Perú",Apartamento Estándar,2 camas dobles,S/ 115,2 adultos · 0 niños · 1 habitación,El Furnished Apartments Arequipa está situado ...
4,Hermoso departamento en exclusivo Condominio,"LOS PARQUES CASA CLUB CALLE HERNANDO DE LUQUE,...",Apartamento de 1 dormitorio,1 cama doble extragrande,S/ 109,2 adultos · 0 niños · 1 habitación,El Hermoso departamento en exclusivo Condomini...
5,Ayli Apart Hotel,"Urb. Monte Bello E-5 Cerro Colorado, Arequipa,...",Apartamento - 2 niveles,1 cama doble grande,S/ 728,2 adultos · 0 niños · 1 habitación,El Ayli Apart Hotel es un apartamento sostenib...
6,"MINI DEPARTAMENTO independiente, privado y cómodo","646 Calle Alfonso Ugarte, Arequipa, Perú",Apartamento de 1 dormitorio,1 cama individual,S/ 131,2 adultos · 0 niños · 1 habitación,"El MINI DEPARTAMENTO independiente, privado y ..."
7,Elegante Apartamento céntrico Arequipa,"249 Juan Velasco Alvarado, Arequipa, Perú",Apartamento de 1 dormitorio,1 cama individual,S/ 172,2 adultos · 0 niños · 1 habitación,Elegante Apartamento céntrico Arequipa is situ...
8,Departamento Amoblado Arequipa,"CALLE AYACUCHO 118 INTERIOR 4, Arequipa, Perú",Apartamento de 3 dormitorios,1 cama doble,S/ 192,2 adultos · 0 niños · 1 habitación,El Departamento Amoblado Arequipa se encuentra...
9,COLCA IMPRESIONANTE APARTAMENTO INDEPENDIENTE,"857 Sepúlveda, Arequipa, Perú",Apartamento de 1 dormitorio,1 cama doble,S/ 134,2 adultos · 0 niños · 1 habitación,El COLCA IMPRESIONANTE APARTAMENTO INDEPENDIEN...


In [18]:
#Primero eliminaremos espacios innecesarios en el dataframe
data_booking = data_booking.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

#Luego eliminaremos la abreviatura de soles para que la data se pueda tratar como número.
data_booking['Precio'] = data_booking['Precio'].str.replace('S/','')

#Por último vamos a cambiar el tipo de dato de Precio a int
data_booking['Precio'] = data_booking['Precio'].astype(int)

#Visualizamos el dataframe ya limpio.
print(data_booking.dtypes)
data_booking

Detalle        object
Direccion      object
Dormitorios    object
Camas          object
Precio          int32
Personas       object
Descripcion    object
dtype: object


,Detalle,Direccion,Dormitorios,Camas,Precio,Personas,Descripcion
0,"Hermoso departamento en el centro ciudad, casa...","420 Melgar, Arequipa, Perú",Apartamento de 2 dormitorios,1 cama individual,134,2 adultos · 0 niños · 1 habitación,"El Hermoso departamento en el centro ciudad, c..."
1,Departamento amplio y bonito en Arequipa 2do Piso,"Urbanización Colegio de ingenieros A9, Cerro C...",Apartamento de 1 dormitorio,1 cama doble,159,2 adultos · 0 niños · 1 habitación,El Departamento Amoblado en Urbanización Coleg...
2,Big apartment near historical center,"506 Lima, Arequipa, Perú",Apartamento,1 cama doble grande,179,2 adultos · 0 niños · 1 habitación,El Big apartment near historical center está s...
3,Furnished Aparments Arequipa,"Urbanización Campiña Dorada C - 1, Arequipa, Perú",Apartamento Estándar,2 camas dobles,115,2 adultos · 0 niños · 1 habitación,El Furnished Apartments Arequipa está situado ...
4,Hermoso departamento en exclusivo Condominio,"LOS PARQUES CASA CLUB CALLE HERNANDO DE LUQUE,...",Apartamento de 1 dormitorio,1 cama doble extragrande,109,2 adultos · 0 niños · 1 habitación,El Hermoso departamento en exclusivo Condomini...
5,Ayli Apart Hotel,"Urb. Monte Bello E-5 Cerro Colorado, Arequipa,...",Apartamento - 2 niveles,1 cama doble grande,728,2 adultos · 0 niños · 1 habitación,El Ayli Apart Hotel es un apartamento sostenib...
6,"MINI DEPARTAMENTO independiente, privado y cómodo","646 Calle Alfonso Ugarte, Arequipa, Perú",Apartamento de 1 dormitorio,1 cama individual,131,2 adultos · 0 niños · 1 habitación,"El MINI DEPARTAMENTO independiente, privado y ..."
7,Elegante Apartamento céntrico Arequipa,"249 Juan Velasco Alvarado, Arequipa, Perú",Apartamento de 1 dormitorio,1 cama individual,172,2 adultos · 0 niños · 1 habitación,Elegante Apartamento céntrico Arequipa is situ...
8,Departamento Amoblado Arequipa,"CALLE AYACUCHO 118 INTERIOR 4, Arequipa, Perú",Apartamento de 3 dormitorios,1 cama doble,192,2 adultos · 0 niños · 1 habitación,El Departamento Amoblado Arequipa se encuentra...
9,COLCA IMPRESIONANTE APARTAMENTO INDEPENDIENTE,"857 Sepúlveda, Arequipa, Perú",Apartamento de 1 dormitorio,1 cama doble,134,2 adultos · 0 niños · 1 habitación,El COLCA IMPRESIONANTE APARTAMENTO INDEPENDIEN...


In [19]:
#Necesitamos obtener el distrito en el cual se encuentran todas las direcciones de los apartamentos
# Lista de distritos de Arequipa, excepto el distrito AQP.
distritos_arequipa = [
    'Cayma', 'Yanahuara', 'Cerro Colorado', 'Tiabaya',
    'Sachaca', 'Miraflores', 'Alto Selva Alegre', 'José Luis Bustamante y Rivero',
    'Hunter', 'Mariano Melgar', 'Paucarpata'
]

# Función para buscar el distrito en la dirección (insensible a mayúsculas y minúsculas)
def buscar_distrito(direccion):
    direccion_lower = direccion.lower()  # Convertir la dirección a minúsculas
    for distrito in distritos_arequipa:
        if distrito.lower() in direccion_lower:  # Convertir el distrito a minúsculas para la comparación
            return distrito
    return None
#

# Crear la columna 'Distrito' usando apply y la función buscar_distrito
data_booking['Distrito'] = data_booking['Direccion'].apply(lambda x: buscar_distrito(x))
#Visualizamos en el dataframe las direcciones que no cuentan con distrito
data_booking[data_booking['Distrito'].isnull()][['Distrito', 'Direccion']]                                       

,Distrito,Direccion
0,None,"420 Melgar, Arequipa, Perú"
2,None,"506 Lima, Arequipa, Perú"
3,None,"Urbanización Campiña Dorada C - 1, Arequipa, Perú"
4,None,"LOS PARQUES CASA CLUB CALLE HERNANDO DE LUQUE,..."
6,None,"646 Calle Alfonso Ugarte, Arequipa, Perú"
7,None,"249 Juan Velasco Alvarado, Arequipa, Perú"
8,None,"CALLE AYACUCHO 118 INTERIOR 4, Arequipa, Perú"
9,None,"857 Sepúlveda, Arequipa, Perú"
10,None,"Calle Urubamba s/n, Edificio Magnus, Apartment..."
11,None,"232 Beaterio 3, Arequipa, Perú"


In [20]:
#Vamos a reemplazar las direcciones
#En primer lugar añadimos manualmente a la siguiente direccion "Urb. Alto de la Luna H2-8, Bustamante y Rivero 2do. piso, Arequipa, Perú", su distrito
# Añadir manualmente el distrito a la dirección específica
direccion_especifica = "Urb. Alto de la Luna H2-8, Bustamante y Rivero 2do. piso, Arequipa, Perú"
distrito_manual = "José Luis Bustamante y Rivero"

# Buscar la dirección específica y asignarle manualmente el distrito
data_booking.loc[data_booking['Direccion'] == direccion_especifica, 'Distrito'] = distrito_manual
# Reemplazamos los valores en blanco en la columna 'Distrito' por 'Arequipa'
data_booking['Distrito'].fillna('Arequipa', inplace=True)
# Añadir ", Arequipa" a todos los valores de la columna 'Distrito'
data_booking['Distrito'] = data_booking['Distrito'].apply(lambda x: x + ', Arequipa' if pd.notnull(x) else x)

data_booking [['Distrito', 'Direccion']]

,Distrito,Direccion
0,"Arequipa, Arequipa","420 Melgar, Arequipa, Perú"
1,"Cerro Colorado, Arequipa","Urbanización Colegio de ingenieros A9, Cerro C..."
2,"Arequipa, Arequipa","506 Lima, Arequipa, Perú"
3,"Arequipa, Arequipa","Urbanización Campiña Dorada C - 1, Arequipa, Perú"
4,"Arequipa, Arequipa","LOS PARQUES CASA CLUB CALLE HERNANDO DE LUQUE,..."
5,"Cerro Colorado, Arequipa","Urb. Monte Bello E-5 Cerro Colorado, Arequipa,..."
6,"Arequipa, Arequipa","646 Calle Alfonso Ugarte, Arequipa, Perú"
7,"Arequipa, Arequipa","249 Juan Velasco Alvarado, Arequipa, Perú"
8,"Arequipa, Arequipa","CALLE AYACUCHO 118 INTERIOR 4, Arequipa, Perú"
9,"Arequipa, Arequipa","857 Sepúlveda, Arequipa, Perú"


In [21]:
#Procedemos a importar ahora la data de la pagina vrbo
data_vrbo = pd.read_excel('Data Vrbo.xlsx')

#Visualizamos el dataframe y el tipo de datos que contiene
print(data_vrbo.dtypes)
data_vrbo

Detalle         object
Direccion       object
Dormitorios     object
Camas           object
Precio          object
Personas        object
Descripcion    float64
dtype: object


,Detalle,Direccion,Dormitorios,Camas,Precio,Personas,Descripcion
0,"Casa Colonial en centro histórico ,excelente u...",Monasterio de Santa Catalina,1 habitación (capacidad para 34 personas),4 camas individuales,37 €,(capacidad para 34 personas),NaN
1,Hermoso departamento en el centro ciudad,Monasterio de Santa Catalina,2 habitaciones (capacidad para 5 personas),1 cama doble,57 €,(capacidad para 5 personas),NaN
2,Hotel turístico a una cuadra de la Plaza de Armas,Catedral de Arequipa,10 habitaciones (capacidad para 25 personas),NaN,33 €,(capacidad para 25 personas),NaN
3,Lovely 3bedroom apt infront Sta Catalina Monas...,Monasterio de Santa Catalina,3 habitaciones (capacidad para 7 personas),1 cama de matrimonio,72 €,(capacidad para 7 personas),NaN
4,Large 2 BR Apartment - Historical Neighborhood...,Monasterio de Santa Catalina,2 habitaciones (capacidad para 4 personas),1 cama de matrimonio,55 €,(capacidad para 4 personas),NaN
5,Casabonne lista para ti,Monasterio de Santa Catalina,2 habitaciones (capacidad para 2 personas),NaN,91 €,(capacidad para 2 personas),NaN
6,Bed & Breakfast Cleofé ¡A 4 cuadras de la pla...,Monasterio de Santa Catalina,9 habitaciones (capacidad para 22 personas),1 cama de matrimonio,42 €,(capacidad para 22 personas),NaN
7,Hostal Premier Arequipa - Habitación Familiar,Plaza de Armas de Arequipa,2 habitaciones (capacidad para 4 personas),2 camas dobles,46 €,(capacidad para 4 personas),NaN
8,¡Cómodo mini apartamento en pleno centro de la...,Plaza de Armas de Arequipa,1 habitación (capacidad para 2 personas),1 cama doble,45 €,(capacidad para 2 personas),NaN
9,¡Cómodo Mini apartamento en pleno centro de la...,Plaza de Armas de Arequipa,1 habitación (capacidad para 2 personas),1 cama de matrimonio grande,47 €,(capacidad para 2 personas),NaN


In [22]:
#Primero eliminaremos espacios innecesarios en el dataframe
data_vrbo = data_vrbo.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

#Luego eliminaremos la abreviatura de soles para que la data se pueda tratar como número.
data_vrbo['Precio'] = data_vrbo['Precio'].str.replace('€','')

#Por último vamos a cambiar el tipo de dato de Precio a int
data_vrbo['Precio'] = data_vrbo['Precio'].astype(int)

print(data_vrbo.dtypes)
data_vrbo

Detalle         object
Direccion       object
Dormitorios     object
Camas           object
Precio           int32
Personas        object
Descripcion    float64
dtype: object


,Detalle,Direccion,Dormitorios,Camas,Precio,Personas,Descripcion
0,"Casa Colonial en centro histórico ,excelente u...",Monasterio de Santa Catalina,1 habitación (capacidad para 34 personas),4 camas individuales,37,(capacidad para 34 personas),NaN
1,Hermoso departamento en el centro ciudad,Monasterio de Santa Catalina,2 habitaciones (capacidad para 5 personas),1 cama doble,57,(capacidad para 5 personas),NaN
2,Hotel turístico a una cuadra de la Plaza de Armas,Catedral de Arequipa,10 habitaciones (capacidad para 25 personas),NaN,33,(capacidad para 25 personas),NaN
3,Lovely 3bedroom apt infront Sta Catalina Monas...,Monasterio de Santa Catalina,3 habitaciones (capacidad para 7 personas),1 cama de matrimonio,72,(capacidad para 7 personas),NaN
4,Large 2 BR Apartment - Historical Neighborhood...,Monasterio de Santa Catalina,2 habitaciones (capacidad para 4 personas),1 cama de matrimonio,55,(capacidad para 4 personas),NaN
5,Casabonne lista para ti,Monasterio de Santa Catalina,2 habitaciones (capacidad para 2 personas),NaN,91,(capacidad para 2 personas),NaN
6,Bed & Breakfast Cleofé ¡A 4 cuadras de la pla...,Monasterio de Santa Catalina,9 habitaciones (capacidad para 22 personas),1 cama de matrimonio,42,(capacidad para 22 personas),NaN
7,Hostal Premier Arequipa - Habitación Familiar,Plaza de Armas de Arequipa,2 habitaciones (capacidad para 4 personas),2 camas dobles,46,(capacidad para 4 personas),NaN
8,¡Cómodo mini apartamento en pleno centro de la...,Plaza de Armas de Arequipa,1 habitación (capacidad para 2 personas),1 cama doble,45,(capacidad para 2 personas),NaN
9,¡Cómodo Mini apartamento en pleno centro de la...,Plaza de Armas de Arequipa,1 habitación (capacidad para 2 personas),1 cama de matrimonio grande,47,(capacidad para 2 personas),NaN


In [23]:
#Primero procederemos a extraer el tipo de cambio de la web y guardarlo como eur_to_pen
api_key = 'TU_CLAVE_DE_API'

url = f"https://open.er-api.com/v6/latest/EUR?apikey={api_key}"

response = requests.get(url)
data = response.json()

if response.status_code == 200:
    eur_to_pen = data['rates']['PEN']
    print(f"El tipo de cambio de EUR a PEN es: {eur_to_pen}")
else:
    print("No se pudo obtener el tipo de cambio.")

El tipo de cambio de EUR a PEN es: 4.078181


In [24]:
#Ahora procedermos a convertir los precios de la data de vrbo
# Función para convertir los precios de euros a soles peruanos
def convert_to_pen(precio_eur, eur_to_pen):
    return precio_eur * eur_to_pen

# Aplicar la conversión a la columna 'Precio_EUR' y crear una nueva columna 'Precio_PEN'
data_vrbo['Precio_PEN'] = data_vrbo['Precio'].apply(lambda x: convert_to_pen(x, eur_to_pen))
# Redondear la columna 'Precio_PEN' a 0 decimales y convertir a enteros
data_vrbo['Precio_PEN'] = data_vrbo['Precio_PEN'].round(0).astype(int)

data_vrbo

,Detalle,Direccion,Dormitorios,Camas,Precio,Personas,Descripcion,Precio_PEN
0,"Casa Colonial en centro histórico ,excelente u...",Monasterio de Santa Catalina,1 habitación (capacidad para 34 personas),4 camas individuales,37,(capacidad para 34 personas),NaN,151
1,Hermoso departamento en el centro ciudad,Monasterio de Santa Catalina,2 habitaciones (capacidad para 5 personas),1 cama doble,57,(capacidad para 5 personas),NaN,232
2,Hotel turístico a una cuadra de la Plaza de Armas,Catedral de Arequipa,10 habitaciones (capacidad para 25 personas),NaN,33,(capacidad para 25 personas),NaN,135
3,Lovely 3bedroom apt infront Sta Catalina Monas...,Monasterio de Santa Catalina,3 habitaciones (capacidad para 7 personas),1 cama de matrimonio,72,(capacidad para 7 personas),NaN,294
4,Large 2 BR Apartment - Historical Neighborhood...,Monasterio de Santa Catalina,2 habitaciones (capacidad para 4 personas),1 cama de matrimonio,55,(capacidad para 4 personas),NaN,224
5,Casabonne lista para ti,Monasterio de Santa Catalina,2 habitaciones (capacidad para 2 personas),NaN,91,(capacidad para 2 personas),NaN,371
6,Bed & Breakfast Cleofé ¡A 4 cuadras de la pla...,Monasterio de Santa Catalina,9 habitaciones (capacidad para 22 personas),1 cama de matrimonio,42,(capacidad para 22 personas),NaN,171
7,Hostal Premier Arequipa - Habitación Familiar,Plaza de Armas de Arequipa,2 habitaciones (capacidad para 4 personas),2 camas dobles,46,(capacidad para 4 personas),NaN,188
8,¡Cómodo mini apartamento en pleno centro de la...,Plaza de Armas de Arequipa,1 habitación (capacidad para 2 personas),1 cama doble,45,(capacidad para 2 personas),NaN,184
9,¡Cómodo Mini apartamento en pleno centro de la...,Plaza de Armas de Arequipa,1 habitación (capacidad para 2 personas),1 cama de matrimonio grande,47,(capacidad para 2 personas),NaN,192


In [25]:
# Ahora que ya tenemos nuestra columna 'Precio_PEN' en el DataFrame data_vrbo procedermos a colocarla en el lugar donde se encontraba precio

# Eliminar la columna 'Precio', que contiene los precios en euros
data_vrbo.drop('Precio', axis=1, inplace=True)

# Moveremos ahora el precio en soles a la columna donde estaba el precio en euros
column_to_move = 'Precio_PEN'
new_position = 4  

# Guardar la columna 'Precio_PEN' en una variable
column = data_vrbo[column_to_move]

# Eliminar la columna 'Precio_PEN' del DataFrame
data_vrbo.drop(column_to_move, axis=1, inplace=True)

# Insertar la columna 'Precio_PEN' en la nueva posición con el nombre de 'Precio'
data_vrbo.insert(new_position, 'Precio', column)

# Mostrar el DataFrame resultante con los precios convertidos a soles peruanos y la columna 'Precio_PEN' movida a la nueva posición, ya con el nombre de 'Precio'
data_vrbo

,Detalle,Direccion,Dormitorios,Camas,Precio,Personas,Descripcion
0,"Casa Colonial en centro histórico ,excelente u...",Monasterio de Santa Catalina,1 habitación (capacidad para 34 personas),4 camas individuales,151,(capacidad para 34 personas),NaN
1,Hermoso departamento en el centro ciudad,Monasterio de Santa Catalina,2 habitaciones (capacidad para 5 personas),1 cama doble,232,(capacidad para 5 personas),NaN
2,Hotel turístico a una cuadra de la Plaza de Armas,Catedral de Arequipa,10 habitaciones (capacidad para 25 personas),NaN,135,(capacidad para 25 personas),NaN
3,Lovely 3bedroom apt infront Sta Catalina Monas...,Monasterio de Santa Catalina,3 habitaciones (capacidad para 7 personas),1 cama de matrimonio,294,(capacidad para 7 personas),NaN
4,Large 2 BR Apartment - Historical Neighborhood...,Monasterio de Santa Catalina,2 habitaciones (capacidad para 4 personas),1 cama de matrimonio,224,(capacidad para 4 personas),NaN
5,Casabonne lista para ti,Monasterio de Santa Catalina,2 habitaciones (capacidad para 2 personas),NaN,371,(capacidad para 2 personas),NaN
6,Bed & Breakfast Cleofé ¡A 4 cuadras de la pla...,Monasterio de Santa Catalina,9 habitaciones (capacidad para 22 personas),1 cama de matrimonio,171,(capacidad para 22 personas),NaN
7,Hostal Premier Arequipa - Habitación Familiar,Plaza de Armas de Arequipa,2 habitaciones (capacidad para 4 personas),2 camas dobles,188,(capacidad para 4 personas),NaN
8,¡Cómodo mini apartamento en pleno centro de la...,Plaza de Armas de Arequipa,1 habitación (capacidad para 2 personas),1 cama doble,184,(capacidad para 2 personas),NaN
9,¡Cómodo Mini apartamento en pleno centro de la...,Plaza de Armas de Arequipa,1 habitación (capacidad para 2 personas),1 cama de matrimonio grande,192,(capacidad para 2 personas),NaN


In [26]:
#Ahora procederemos a añadir el distrito a cada direccion
direccion_especifica = [
    "Monasterio de Santa Catalina",
    "Catedral de Arequipa",
    "Plaza de Armas de Arequipa",
    "Plaza de Yanahuara",
    "Puente Grau"
]
distrito_manual = ["Arequipa", "Arequipa", "Arequipa", "Yanahuara", "Yanahuara"]

# Iterar sobre las listas y reemplazar los valores en el DataFrame
for i, direccion in enumerate(direccion_especifica):
    data_vrbo.loc[data_vrbo['Direccion'] == direccion, 'Distrito'] = distrito_manual[i]
#Visualizamos las columnas en el dataframe
data_vrbo[['Distrito', 'Direccion']]

,Distrito,Direccion
0,Arequipa,Monasterio de Santa Catalina
1,Arequipa,Monasterio de Santa Catalina
2,Arequipa,Catedral de Arequipa
3,Arequipa,Monasterio de Santa Catalina
4,Arequipa,Monasterio de Santa Catalina
5,Arequipa,Monasterio de Santa Catalina
6,Arequipa,Monasterio de Santa Catalina
7,Arequipa,Plaza de Armas de Arequipa
8,Arequipa,Plaza de Armas de Arequipa
9,Arequipa,Plaza de Armas de Arequipa


In [27]:
# Por último, procedemos a añadir ", Arequipa" a todos los valores de la columna 'Distrito'
data_vrbo['Distrito'] = data_vrbo['Distrito'].apply(lambda x: x + ', Arequipa' if pd.notnull(x) else x)
data_vrbo[['Distrito', 'Direccion']]

,Distrito,Direccion
0,"Arequipa, Arequipa",Monasterio de Santa Catalina
1,"Arequipa, Arequipa",Monasterio de Santa Catalina
2,"Arequipa, Arequipa",Catedral de Arequipa
3,"Arequipa, Arequipa",Monasterio de Santa Catalina
4,"Arequipa, Arequipa",Monasterio de Santa Catalina
5,"Arequipa, Arequipa",Monasterio de Santa Catalina
6,"Arequipa, Arequipa",Monasterio de Santa Catalina
7,"Arequipa, Arequipa",Plaza de Armas de Arequipa
8,"Arequipa, Arequipa",Plaza de Armas de Arequipa
9,"Arequipa, Arequipa",Plaza de Armas de Arequipa


In [28]:
#Importamos el archivo de excel de la data de Expedia
data_expedia = pd.read_excel('Data Expedia.xlsx')

#Visualizamos el dataframe y el tipo de datos que contiene
print(data_expedia.dtypes)
data_expedia

Detalle        object
Direccion      object
Dormitorios    object
Precio         object
dtype: object


,Detalle,Direccion,Dormitorios,Precio
0,¡Cómodo mini apartamento en pleno centro de la...,Centro histórico,"2 huéspedes, 1 habitación, 1 baño",45 €
1,518 Jerusalén st,Centro histórico,Apartamento entero,36 €
2,Duplex 2 Habitaciones,Paucarpata,"4 huéspedes, 2 habitaciones, 2 baños",59 €
3,Departamento amplio y bonito en Arequipa 2do Piso,Distrito de Cerro Colorado,"10 huéspedes, 3 habitaciones, 2 baños",54 €
4,My Sweet House,Arequipa,Apartamento entero,32 €
5,Studio suite for 2 people at Central City Cent...,Centro histórico,"2 huéspedes, 1 habitación, 1 baño",37 €
6,Hostal Premier Arequipa - Habitación Triple,Arequipa,"3 huéspedes, 2 habitaciones, 2 baños",46 €
7,Hermoso departamento en el centro ciudad,Centro histórico,"5 huéspedes, 2 habitaciones, 2 baños",57 €
8,¡Cómodo Mini apartamento en pleno centro de la...,Centro histórico,"2 huéspedes, 1 habitación, 1 baño",47 €
9,Beautiful and Modern Apartment,NaN,NaN,77 €


In [29]:
#Primero eliminaremos espacios innecesarios en el dataframe
data_expedia = data_expedia.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

#Luego eliminaremos la abreviatura de soles para que la data se pueda tratar como número.
data_expedia['Precio'] = data_expedia['Precio'].str.replace('€','')

#Por último vamos a cambiar el tipo de dato de Precio a int
data_expedia['Precio'] = data_expedia['Precio'].astype(int)

#Visualizamos el dataframe ya limpio.
print(data_expedia.dtypes)
data_expedia

Detalle        object
Direccion      object
Dormitorios    object
Precio          int32
dtype: object


,Detalle,Direccion,Dormitorios,Precio
0,¡Cómodo mini apartamento en pleno centro de la...,Centro histórico,"2 huéspedes, 1 habitación, 1 baño",45
1,518 Jerusalén st,Centro histórico,Apartamento entero,36
2,Duplex 2 Habitaciones,Paucarpata,"4 huéspedes, 2 habitaciones, 2 baños",59
3,Departamento amplio y bonito en Arequipa 2do Piso,Distrito de Cerro Colorado,"10 huéspedes, 3 habitaciones, 2 baños",54
4,My Sweet House,Arequipa,Apartamento entero,32
5,Studio suite for 2 people at Central City Cent...,Centro histórico,"2 huéspedes, 1 habitación, 1 baño",37
6,Hostal Premier Arequipa - Habitación Triple,Arequipa,"3 huéspedes, 2 habitaciones, 2 baños",46
7,Hermoso departamento en el centro ciudad,Centro histórico,"5 huéspedes, 2 habitaciones, 2 baños",57
8,¡Cómodo Mini apartamento en pleno centro de la...,Centro histórico,"2 huéspedes, 1 habitación, 1 baño",47
9,Beautiful and Modern Apartment,NaN,NaN,77


In [30]:
# Aplicar la conversión a la columna 'Precio_EUR' y crear una nueva columna 'Precio_PEN'
data_expedia['Precio_PEN'] = data_expedia['Precio'].apply(lambda x: convert_to_pen(x, eur_to_pen))
# Redondear la columna 'Precio_PEN' a 0 decimales y convertir a enteros
data_expedia['Precio_PEN'] = data_expedia['Precio_PEN'].round(0).astype(int)

data_expedia

,Detalle,Direccion,Dormitorios,Precio,Precio_PEN
0,¡Cómodo mini apartamento en pleno centro de la...,Centro histórico,"2 huéspedes, 1 habitación, 1 baño",45,184
1,518 Jerusalén st,Centro histórico,Apartamento entero,36,147
2,Duplex 2 Habitaciones,Paucarpata,"4 huéspedes, 2 habitaciones, 2 baños",59,241
3,Departamento amplio y bonito en Arequipa 2do Piso,Distrito de Cerro Colorado,"10 huéspedes, 3 habitaciones, 2 baños",54,220
4,My Sweet House,Arequipa,Apartamento entero,32,131
5,Studio suite for 2 people at Central City Cent...,Centro histórico,"2 huéspedes, 1 habitación, 1 baño",37,151
6,Hostal Premier Arequipa - Habitación Triple,Arequipa,"3 huéspedes, 2 habitaciones, 2 baños",46,188
7,Hermoso departamento en el centro ciudad,Centro histórico,"5 huéspedes, 2 habitaciones, 2 baños",57,232
8,¡Cómodo Mini apartamento en pleno centro de la...,Centro histórico,"2 huéspedes, 1 habitación, 1 baño",47,192
9,Beautiful and Modern Apartment,NaN,NaN,77,314


In [31]:
# Ahora que ya tenemos nuestra columna 'Precio_PEN' en el DataFrame data_vrbo procedermos a colocarla en el lugar donde se encontraba precio

# Eliminar la columna 'Precio', que contiene los precios en euros
data_expedia.drop('Precio', axis=1, inplace=True)

# Moveremos ahora el precio en soles a la columna donde estaba el precio en euros
column_to_move = 'Precio_PEN'
new_position = 3  

# Guardar la columna 'Precio_PEN' en una variable
column = data_expedia[column_to_move]

# Eliminar la columna 'Precio_PEN' del DataFrame
data_expedia.drop(column_to_move, axis=1, inplace=True)

# Insertar la columna 'Precio_PEN' en la nueva posición con el nombre de 'Precio'
data_expedia.insert(new_position, 'Precio', column)

# Mostrar el DataFrame resultante con los precios convertidos a soles peruanos y la columna 'Precio_PEN' movida a la nueva posición, ya con el nombre de 'Precio'
data_expedia

,Detalle,Direccion,Dormitorios,Precio
0,¡Cómodo mini apartamento en pleno centro de la...,Centro histórico,"2 huéspedes, 1 habitación, 1 baño",184
1,518 Jerusalén st,Centro histórico,Apartamento entero,147
2,Duplex 2 Habitaciones,Paucarpata,"4 huéspedes, 2 habitaciones, 2 baños",241
3,Departamento amplio y bonito en Arequipa 2do Piso,Distrito de Cerro Colorado,"10 huéspedes, 3 habitaciones, 2 baños",220
4,My Sweet House,Arequipa,Apartamento entero,131
5,Studio suite for 2 people at Central City Cent...,Centro histórico,"2 huéspedes, 1 habitación, 1 baño",151
6,Hostal Premier Arequipa - Habitación Triple,Arequipa,"3 huéspedes, 2 habitaciones, 2 baños",188
7,Hermoso departamento en el centro ciudad,Centro histórico,"5 huéspedes, 2 habitaciones, 2 baños",232
8,¡Cómodo Mini apartamento en pleno centro de la...,Centro histórico,"2 huéspedes, 1 habitación, 1 baño",192
9,Beautiful and Modern Apartment,NaN,NaN,314


In [32]:
#Ahora procederemos a añadir el distrito de manera manual a los hospedajes que estan vacios y con error
direccion_especifica = [
    "Centro histórico",
    "6 huéspedes, 3 habitaciones, 1 baño",
    "Paucarpata",
    "Distrito de Cerro Colorado",
    "Arequipa",
]

distrito_manual = ["Arequipa", "Arequipa", "Paucarpata", "Cerro Colorado", "Arequipa"]

# Iterar sobre las listas y reemplazar los valores en el DataFrame
for i, direccion in enumerate(direccion_especifica):
    data_expedia.loc[data_expedia['Direccion'] == direccion, 'Distrito'] = distrito_manual[i]
#Para los valores en blanco, utilizaremos la siguiente funcion
direccion_manual = "Arequipa"
data_expedia = data_expedia.fillna(direccion_manual)
#Visualizamos las columnas en el dataframe
data_expedia[['Distrito', 'Direccion']]

,Distrito,Direccion
0,Arequipa,Centro histórico
1,Arequipa,Centro histórico
2,Paucarpata,Paucarpata
3,Cerro Colorado,Distrito de Cerro Colorado
4,Arequipa,Arequipa
5,Arequipa,Centro histórico
6,Arequipa,Arequipa
7,Arequipa,Centro histórico
8,Arequipa,Centro histórico
9,Arequipa,Arequipa


In [39]:
# Vamos a modificar esa direccion erronea.
# Buscar y reemplazar coincidencias parciales basadas en "huéspedes"
data_expedia['Direccion'] = data_expedia['Direccion'].str.replace(r'\b\d+\s*huéspedes\b.*', 'Centro histórico', regex=True)

# Verificamos el DataFrame actualizado
data_expedia[['Distrito', 'Direccion']]

,Distrito,Direccion
0,Arequipa,Centro histórico
1,Arequipa,Centro histórico
2,Paucarpata,Paucarpata
3,Cerro Colorado,Distrito de Cerro Colorado
4,Arequipa,Arequipa
5,Arequipa,Centro histórico
6,Arequipa,Arequipa
7,Arequipa,Centro histórico
8,Arequipa,Centro histórico
9,Arequipa,Arequipa


In [40]:
# Por último, procedemos a añadir ", Arequipa" a todos los valores de la columna 'Distrito'
data_expedia['Distrito'] = data_expedia['Distrito'].apply(lambda x: x + ', Arequipa' if pd.notnull(x) else x)
data_expedia[['Distrito', 'Direccion']]

,Distrito,Direccion
0,"Arequipa, Arequipa",Centro histórico
1,"Arequipa, Arequipa",Centro histórico
2,"Paucarpata, Arequipa",Paucarpata
3,"Cerro Colorado, Arequipa",Distrito de Cerro Colorado
4,"Arequipa, Arequipa",Arequipa
5,"Arequipa, Arequipa",Centro histórico
6,"Arequipa, Arequipa",Arequipa
7,"Arequipa, Arequipa",Centro histórico
8,"Arequipa, Arequipa",Centro histórico
9,"Arequipa, Arequipa",Arequipa


In [41]:
# Concatenar los DataFrames uno encima del otro
data_airbnb = pd.concat([data_booking, data_vrbo, data_expedia])

#Vamos a añadir una columna con el tipo de alojamiento
tipo_alojamiento = 'Airbnb'

data_airbnb['Tipo']= tipo_alojamiento

#Eliminamos los espacios innecesarios en el dataframe
data_airbnb = data_airbnb.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

# Mostrar el DataFrame resultante
print(data_airbnb.dtypes)
data_airbnb

Detalle        object
Direccion      object
Dormitorios    object
Camas          object
Precio          int32
Personas       object
Descripcion    object
Distrito       object
Tipo           object
dtype: object


,Detalle,Direccion,Dormitorios,Camas,Precio,Personas,Descripcion,Distrito,Tipo
0,"Hermoso departamento en el centro ciudad, casa...","420 Melgar, Arequipa, Perú",Apartamento de 2 dormitorios,1 cama individual,134,2 adultos · 0 niños · 1 habitación,"El Hermoso departamento en el centro ciudad, c...","Arequipa, Arequipa",Airbnb
1,Departamento amplio y bonito en Arequipa 2do Piso,"Urbanización Colegio de ingenieros A9, Cerro C...",Apartamento de 1 dormitorio,1 cama doble,159,2 adultos · 0 niños · 1 habitación,El Departamento Amoblado en Urbanización Coleg...,"Cerro Colorado, Arequipa",Airbnb
2,Big apartment near historical center,"506 Lima, Arequipa, Perú",Apartamento,1 cama doble grande,179,2 adultos · 0 niños · 1 habitación,El Big apartment near historical center está s...,"Arequipa, Arequipa",Airbnb
3,Furnished Aparments Arequipa,"Urbanización Campiña Dorada C - 1, Arequipa, Perú",Apartamento Estándar,2 camas dobles,115,2 adultos · 0 niños · 1 habitación,El Furnished Apartments Arequipa está situado ...,"Arequipa, Arequipa",Airbnb
4,Hermoso departamento en exclusivo Condominio,"LOS PARQUES CASA CLUB CALLE HERNANDO DE LUQUE,...",Apartamento de 1 dormitorio,1 cama doble extragrande,109,2 adultos · 0 niños · 1 habitación,El Hermoso departamento en exclusivo Condomini...,"Arequipa, Arequipa",Airbnb
...,...,...,...,...,...,...,...,...,...
6,Hostal Premier Arequipa - Habitación Triple,Arequipa,"3 huéspedes, 2 habitaciones, 2 baños",NaN,188,NaN,NaN,"Arequipa, Arequipa",Airbnb
7,Hermoso departamento en el centro ciudad,Centro histórico,"5 huéspedes, 2 habitaciones, 2 baños",NaN,232,NaN,NaN,"Arequipa, Arequipa",Airbnb
8,¡Cómodo Mini apartamento en pleno centro de la...,Centro histórico,"2 huéspedes, 1 habitación, 1 baño",NaN,192,NaN,NaN,"Arequipa, Arequipa",Airbnb
9,Beautiful and Modern Apartment,Arequipa,Arequipa,NaN,314,NaN,NaN,"Arequipa, Arequipa",Airbnb


In [42]:
#Procederemos ahora a extraer las coordenadas de los distritos de cada direccion
# Utilizamos geolocator con un user agent cualquiera
geolocator = Nominatim(user_agent='Arequipa-Peru')

# Utilizamos geocode para extraer la location basada en el distrito
start = time.time()

# Geocode
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

# Establecemos la nueva columna de location
data_airbnb['Location'] = data_airbnb['Distrito'].apply(geocode)

# Una vez encontrada la location, extraemos las coordenadas en columnas separadas
data_airbnb['Latitud'] = data_airbnb['Location'].apply(lambda x: x.latitude if x else None)
data_airbnb['Longitud'] = data_airbnb['Location'].apply(lambda x: x.longitude if x else None)

# Finalizamos la función de tiempo
end = time.time()
elapsed = end - start

# Visualizamos el dataframe
print(str(elapsed) + " segundos")
# data_airbnb.drop('Location', axis=1, inplace=True)  # Opcional: eliminar la columna 'Location' si ya no es necesaria
data_airbnb

77.0098524093628 segundos


,Detalle,Direccion,Dormitorios,Camas,Precio,Personas,Descripcion,Distrito,Tipo,Location,Latitud,Longitud
0,"Hermoso departamento en el centro ciudad, casa...","420 Melgar, Arequipa, Perú",Apartamento de 2 dormitorios,1 cama individual,134,2 adultos · 0 niños · 1 habitación,"El Hermoso departamento en el centro ciudad, c...","Arequipa, Arequipa",Airbnb,"(Arequipa, Perú, (-16.3988667, -71.5369607))",-16.398867,-71.536961
1,Departamento amplio y bonito en Arequipa 2do Piso,"Urbanización Colegio de ingenieros A9, Cerro C...",Apartamento de 1 dormitorio,1 cama doble,159,2 adultos · 0 niños · 1 habitación,El Departamento Amoblado en Urbanización Coleg...,"Cerro Colorado, Arequipa",Airbnb,"(Cerro Colorado, Arequipa, Perú, (-16.29799905...",-16.297999,-71.573948
2,Big apartment near historical center,"506 Lima, Arequipa, Perú",Apartamento,1 cama doble grande,179,2 adultos · 0 niños · 1 habitación,El Big apartment near historical center está s...,"Arequipa, Arequipa",Airbnb,"(Arequipa, Perú, (-16.3988667, -71.5369607))",-16.398867,-71.536961
3,Furnished Aparments Arequipa,"Urbanización Campiña Dorada C - 1, Arequipa, Perú",Apartamento Estándar,2 camas dobles,115,2 adultos · 0 niños · 1 habitación,El Furnished Apartments Arequipa está situado ...,"Arequipa, Arequipa",Airbnb,"(Arequipa, Perú, (-16.3988667, -71.5369607))",-16.398867,-71.536961
4,Hermoso departamento en exclusivo Condominio,"LOS PARQUES CASA CLUB CALLE HERNANDO DE LUQUE,...",Apartamento de 1 dormitorio,1 cama doble extragrande,109,2 adultos · 0 niños · 1 habitación,El Hermoso departamento en exclusivo Condomini...,"Arequipa, Arequipa",Airbnb,"(Arequipa, Perú, (-16.3988667, -71.5369607))",-16.398867,-71.536961
...,...,...,...,...,...,...,...,...,...,...,...,...
6,Hostal Premier Arequipa - Habitación Triple,Arequipa,"3 huéspedes, 2 habitaciones, 2 baños",NaN,188,NaN,NaN,"Arequipa, Arequipa",Airbnb,"(Arequipa, Perú, (-16.3988667, -71.5369607))",-16.398867,-71.536961
7,Hermoso departamento en el centro ciudad,Centro histórico,"5 huéspedes, 2 habitaciones, 2 baños",NaN,232,NaN,NaN,"Arequipa, Arequipa",Airbnb,"(Arequipa, Perú, (-16.3988667, -71.5369607))",-16.398867,-71.536961
8,¡Cómodo Mini apartamento en pleno centro de la...,Centro histórico,"2 huéspedes, 1 habitación, 1 baño",NaN,192,NaN,NaN,"Arequipa, Arequipa",Airbnb,"(Arequipa, Perú, (-16.3988667, -71.5369607))",-16.398867,-71.536961
9,Beautiful and Modern Apartment,Arequipa,Arequipa,NaN,314,NaN,NaN,"Arequipa, Arequipa",Airbnb,"(Arequipa, Perú, (-16.3988667, -71.5369607))",-16.398867,-71.536961


In [43]:
#Procedemos a guardar el dataframe en un archivo xlsx nuevo
nombre_archivo = 'Data Airbnb limpio.xlsx'

data_airbnb.to_excel(nombre_archivo, index=False)  # El argumento index=False evita que se incluyan los índices en el archivo

print(f'DataFrame guardado en {nombre_archivo}')

DataFrame guardado en Data Airbnb limpio.xlsx


In [2]:
#Importamos el archivo de excel
data_adondevivir = pd.read_excel('Data adondevivir.xlsx')

#Visualizamos el dataframe y el tipo de datos que contiene
print(data_adondevivir.dtypes)
data_adondevivir

Detalle        object
Direccion      object
Dormitorios    object
Precio         object
Descripcion    object
dtype: object


,Detalle,Direccion,Dormitorios,Precio,Descripcion
0,Alquilo Depa Amoblado en Vallecito,"Vallecito, Arequipa",3 dorm.,"S/ 2,600",Moderno y amplio departamento totalmente amobl...
1,Alquilo Exelente Departamento Amoblado Cerca a...,"Arequipa, Arequipa",3 dorm.,"S/ 3,500",#Alquilo moderno #Departamento amoblado. Cuent...
2,Alquilo Moderno y Amplio Departamento en Cerca...,"Arequipa, Arequipa",3 dorm.,"S/ 1,600",#Alquilo moderno y amplio #Departamento en #Ce...
3,Hermoso Departamento Amoblado en Umacollo,"Yanahuara, Arequipa",3 dorm.,"S/ 1,700",Ubicado a 06 cuadras de la universidad catolic...
4,Alquilo Comodo Departamento Muy Bien Ubicado e...,"Cayma, Arequipa",3 dorm.,"S/ 1,500",#Alquilo cómodo #Departamento sin amoblar en U...
...,...,...,...,...,...
68,Departamento en Alquiler - Cayma,"Arequipa, Arequipa",3 dorm.,"S/ 3,000",Hermoso departamento de 115 mtr2. Urbanización...
69,Alquilo Acogedor Departamento Amoblado en Urb....,"Cayma, Arequipa",3 dorm.,"S/ 2,500",#Alquilo acogedor #Departamento amoblado en Ur...
70,"Alquiler Minidepartamento Amoblado, Calle Lima...","Yanahuara, Arequipa",2 dorm.,"S/ 1,800",Se alquila departamento amoblado en el segundo...
71,"Oportunidad, en Venta o Alquiler Moderno Duple...","Mejia, Islay",3 dorm.,"S/ 7,500",En Venta o alquiler moderno duplex en Mejia. u...


In [3]:
#Primero eliminaremos espacios innecesarios en el dataframe
data_adondevivir = data_adondevivir.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

#Luego eliminaremos la abreviatura de soles para que la data se pueda tratar como número.
data_adondevivir['Precio'] = data_adondevivir['Precio'].str.replace('S/','')
#Eliminamos las comas de los precios para poder luego convertir el tipo de dato a número entero.
data_adondevivir['Precio'] = data_adondevivir['Precio'].str.replace(',','')
#Despuúes eliminaremos el simbolo # de las descripciones
data_adondevivir['Descripcion'] = data_adondevivir['Descripcion'].str.replace('#','')

#Por último vamos a cambiar el tipo de dato de Precio a int
data_adondevivir['Precio'] = data_adondevivir['Precio'].astype(int)

#Visualizamos el Dataframe ya depurado
print(data_adondevivir.dtypes)
data_adondevivir

Detalle        object
Direccion      object
Dormitorios    object
Precio          int32
Descripcion    object
dtype: object


,Detalle,Direccion,Dormitorios,Precio,Descripcion
0,Alquilo Depa Amoblado en Vallecito,"Vallecito, Arequipa",3 dorm.,2600,Moderno y amplio departamento totalmente amobl...
1,Alquilo Exelente Departamento Amoblado Cerca a...,"Arequipa, Arequipa",3 dorm.,3500,Alquilo moderno Departamento amoblado. Cuenta ...
2,Alquilo Moderno y Amplio Departamento en Cerca...,"Arequipa, Arequipa",3 dorm.,1600,Alquilo moderno y amplio Departamento en Cerca...
3,Hermoso Departamento Amoblado en Umacollo,"Yanahuara, Arequipa",3 dorm.,1700,Ubicado a 06 cuadras de la universidad catolic...
4,Alquilo Comodo Departamento Muy Bien Ubicado e...,"Cayma, Arequipa",3 dorm.,1500,Alquilo cómodo Departamento sin amoblar en Urb...
...,...,...,...,...,...
68,Departamento en Alquiler - Cayma,"Arequipa, Arequipa",3 dorm.,3000,Hermoso departamento de 115 mtr2. Urbanización...
69,Alquilo Acogedor Departamento Amoblado en Urb....,"Cayma, Arequipa",3 dorm.,2500,Alquilo acogedor Departamento amoblado en Urba...
70,"Alquiler Minidepartamento Amoblado, Calle Lima...","Yanahuara, Arequipa",2 dorm.,1800,Se alquila departamento amoblado en el segundo...
71,"Oportunidad, en Venta o Alquiler Moderno Duple...","Mejia, Islay",3 dorm.,7500,En Venta o alquiler moderno duplex en Mejia. u...


In [4]:
#Encontramos que habian datos erroneo en el dataframe, el cual correspondia a apartamentos en venta
#Procedemos a eliminar esas filas
data_adondevivir = data_adondevivir[data_adondevivir['Precio'] <= 9999]
data_adondevivir

,Detalle,Direccion,Dormitorios,Precio,Descripcion
0,Alquilo Depa Amoblado en Vallecito,"Vallecito, Arequipa",3 dorm.,2600,Moderno y amplio departamento totalmente amobl...
1,Alquilo Exelente Departamento Amoblado Cerca a...,"Arequipa, Arequipa",3 dorm.,3500,Alquilo moderno Departamento amoblado. Cuenta ...
2,Alquilo Moderno y Amplio Departamento en Cerca...,"Arequipa, Arequipa",3 dorm.,1600,Alquilo moderno y amplio Departamento en Cerca...
3,Hermoso Departamento Amoblado en Umacollo,"Yanahuara, Arequipa",3 dorm.,1700,Ubicado a 06 cuadras de la universidad catolic...
4,Alquilo Comodo Departamento Muy Bien Ubicado e...,"Cayma, Arequipa",3 dorm.,1500,Alquilo cómodo Departamento sin amoblar en Urb...
...,...,...,...,...,...
68,Departamento en Alquiler - Cayma,"Arequipa, Arequipa",3 dorm.,3000,Hermoso departamento de 115 mtr2. Urbanización...
69,Alquilo Acogedor Departamento Amoblado en Urb....,"Cayma, Arequipa",3 dorm.,2500,Alquilo acogedor Departamento amoblado en Urba...
70,"Alquiler Minidepartamento Amoblado, Calle Lima...","Yanahuara, Arequipa",2 dorm.,1800,Se alquila departamento amoblado en el segundo...
71,"Oportunidad, en Venta o Alquiler Moderno Duple...","Mejia, Islay",3 dorm.,7500,En Venta o alquiler moderno duplex en Mejia. u...


In [5]:
#De igual manera, encontramos que habian departamentos que se encontraban fuera de AQP, por lo que deberian ser eliminadas esas filas.
# Filtrar el DataFrame para mantener solo las filas que contienen 'Arequipa' en la columna 'Direccion' y eliminar el resto
data_adondevivir = data_adondevivir[data_adondevivir['Direccion'].str.contains('Arequipa', case=False, na=False)]

#Visualizamos el dataframe ya limpio
data_adondevivir

,Detalle,Direccion,Dormitorios,Precio,Descripcion
0,Alquilo Depa Amoblado en Vallecito,"Vallecito, Arequipa",3 dorm.,2600,Moderno y amplio departamento totalmente amobl...
1,Alquilo Exelente Departamento Amoblado Cerca a...,"Arequipa, Arequipa",3 dorm.,3500,Alquilo moderno Departamento amoblado. Cuenta ...
2,Alquilo Moderno y Amplio Departamento en Cerca...,"Arequipa, Arequipa",3 dorm.,1600,Alquilo moderno y amplio Departamento en Cerca...
3,Hermoso Departamento Amoblado en Umacollo,"Yanahuara, Arequipa",3 dorm.,1700,Ubicado a 06 cuadras de la universidad catolic...
4,Alquilo Comodo Departamento Muy Bien Ubicado e...,"Cayma, Arequipa",3 dorm.,1500,Alquilo cómodo Departamento sin amoblar en Urb...
5,Alquilo Hermoso y Acogedor Departamento Totalm...,"Cayma, Arequipa",3 dorm.,3500,Alquilo moderno Departamento amoblado en Urban...
6,Alquilo Bonito Departamento Amoblado Exclusivo...,"Cerro Colorado, Arequipa",1 dorm.,1800,(4) piso. (Sin) Ascensor. Área de (60) m2. (1)...
8,Amplio y Hermoso Penthouse en Venta o Alquiler!,"Yanahuara, Arequipa",4 dorm.,4400,En venta O alquiler! Hermoso penthouse tipo du...
10,Departamento Flat - Sachaca,"Sachaca, Arequipa",3 dorm.,2800,Exclusivo departamento de 140mtr2. Condominio ...
11,Alquilo Amplio y Moderno Departamento Completa...,"Cayma, Arequipa",3 dorm.,2800,En alquiler! Amplio y moderno departamento com...


In [6]:
#Procederemos ahora a extraer las coordenadas de de cada direccion
# Utilizamos geolocator con un user agent cualquiera
geolocator = Nominatim(user_agent='Arequipa-Peru')

# Utilizamos geocode para extraer la location basada en el distrito
start = time.time()

# Geocode
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

# Establecemos la nueva columna de location
data_adondevivir['Location'] = data_adondevivir['Direccion'].apply(geocode)

# Una vez encontrada la location, extraemos las coordenadas en columnas separadas
data_adondevivir['Latitud'] = data_adondevivir['Location'].apply(lambda x: x.latitude if x else None)
data_adondevivir['Longitud'] = data_adondevivir['Location'].apply(lambda x: x.longitude if x else None)

# Finalizamos la función de tiempo
end = time.time()
elapsed = end - start

# Visualizamos el dataframe
print(str(elapsed) + " segundos")
#Visualizamos el dataframe
data_adondevivir

48.44125723838806 segundos


C:\Users\Usuario\AppData\Local\Temp\ipykernel_13600\488747987.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_adondevivir['Location'] = data_adondevivir['Direccion'].apply(geocode)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_13600\488747987.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_adondevivir['Latitud'] = data_adondevivir['Location'].apply(lambda x: x.latitude if x else None)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_13600\488747987.py:16: SettingWithCopyWarning: 
A value is

,Detalle,Direccion,Dormitorios,Precio,Descripcion,Location,Latitud,Longitud
0,Alquilo Depa Amoblado en Vallecito,"Vallecito, Arequipa",3 dorm.,2600,Moderno y amplio departamento totalmente amobl...,"(Vallecito, Chuquibamba, Condesuyos, Arequipa,...",-15.842004,-72.662038
1,Alquilo Exelente Departamento Amoblado Cerca a...,"Arequipa, Arequipa",3 dorm.,3500,Alquilo moderno Departamento amoblado. Cuenta ...,"(Arequipa, Perú, (-16.3988667, -71.5369607))",-16.398867,-71.536961
2,Alquilo Moderno y Amplio Departamento en Cerca...,"Arequipa, Arequipa",3 dorm.,1600,Alquilo moderno y amplio Departamento en Cerca...,"(Arequipa, Perú, (-16.3988667, -71.5369607))",-16.398867,-71.536961
3,Hermoso Departamento Amoblado en Umacollo,"Yanahuara, Arequipa",3 dorm.,1700,Ubicado a 06 cuadras de la universidad catolic...,"(Yanahuara, Arequipa, Perú, (-16.38818185, -71...",-16.388182,-71.542054
4,Alquilo Comodo Departamento Muy Bien Ubicado e...,"Cayma, Arequipa",3 dorm.,1500,Alquilo cómodo Departamento sin amoblar en Urb...,"(Cayma, Arequipa, Perú, (-16.27939415, -71.460...",-16.279394,-71.460312
5,Alquilo Hermoso y Acogedor Departamento Totalm...,"Cayma, Arequipa",3 dorm.,3500,Alquilo moderno Departamento amoblado en Urban...,"(Cayma, Arequipa, Perú, (-16.27939415, -71.460...",-16.279394,-71.460312
6,Alquilo Bonito Departamento Amoblado Exclusivo...,"Cerro Colorado, Arequipa",1 dorm.,1800,(4) piso. (Sin) Ascensor. Área de (60) m2. (1)...,"(Cerro Colorado, Arequipa, Perú, (-16.29799905...",-16.297999,-71.573948
8,Amplio y Hermoso Penthouse en Venta o Alquiler!,"Yanahuara, Arequipa",4 dorm.,4400,En venta O alquiler! Hermoso penthouse tipo du...,"(Yanahuara, Arequipa, Perú, (-16.38818185, -71...",-16.388182,-71.542054
10,Departamento Flat - Sachaca,"Sachaca, Arequipa",3 dorm.,2800,Exclusivo departamento de 140mtr2. Condominio ...,"(Sachaca, Arequipa, 04013, Perú, (-16.42560499...",-16.425605,-71.575790
11,Alquilo Amplio y Moderno Departamento Completa...,"Cayma, Arequipa",3 dorm.,2800,En alquiler! Amplio y moderno departamento com...,"(Cayma, Arequipa, Perú, (-16.27939415, -71.460...",-16.279394,-71.460312


In [7]:
#Importamos el archivo de excel
data_properati = pd.read_excel('Data Properati.xlsx')

#Visualizamos el dataframe y el tipo de datos que contiene
print(data_properati.dtypes)
data_properati

Detalle        object
Direccion      object
Dormitorios    object
Precio         object
dtype: object


,Detalle,Direccion,Dormitorios,Precio
0,Apartamento en Alquiler en Retamas,"Retamas, Yanahuara, Arequipa",3 dormitorios,"S/.2,400"
1,Apartamento en Alquiler en Sachaca,"Sachaca, Arequipa",3 dormitorios,"S/.1,600"
2,Apartamento en Alquiler en Vallecito,"Vallecito, Arequipa",3 dormitorios,"S/.2,600"
3,Apartamento en Alquiler en Los Alamos,"Los Alamos, Arequipa",3 dormitorios,"USD1,400"
4,Apartamento en Alquiler en Ur. El Señorial,"Ur. El Señorial, Cayma, Arequipa",3 dormitorios,"USD1,500"
...,...,...,...,...
63,Apartamento en Alquiler en José Luis Bustamant...,"José Luis Bustamante y Rivero, Arequipa",2 dormitorios,"S/.1,500"
64,Apartamento en Alquiler en Miraflores,"Miraflores, Arequipa",3 dormitorios,"USD1,500"
65,Apartamento en Alquiler en Vallecito,"Vallecito, Arequipa",3 dormitorios,"S/.3,300"
66,Apartamento en Alquiler en Cayma,"Cayma, Arequipa",2 dormitorios,"S/.1,450"


In [8]:
#Primero eliminaremos espacios innecesarios en el dataframe
data_properati = data_properati.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

#Luego eliminaremos la abreviatura de soles para que la data se pueda tratar como número.
data_properati['Precio'] = data_properati['Precio'].str.replace('S/.','')
#Eliminamos las comas de los precios para poder luego convertir el tipo de dato a número entero.
data_properati['Precio'] = data_properati['Precio'].str.replace(',','')

#Visualizamos el dataframe
data_properati

C:\Users\Usuario\AppData\Local\Temp\ipykernel_13600\2149337102.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  data_properati['Precio'] = data_properati['Precio'].str.replace('S/.','')


,Detalle,Direccion,Dormitorios,Precio
0,Apartamento en Alquiler en Retamas,"Retamas, Yanahuara, Arequipa",3 dormitorios,2400
1,Apartamento en Alquiler en Sachaca,"Sachaca, Arequipa",3 dormitorios,1600
2,Apartamento en Alquiler en Vallecito,"Vallecito, Arequipa",3 dormitorios,2600
3,Apartamento en Alquiler en Los Alamos,"Los Alamos, Arequipa",3 dormitorios,USD1400
4,Apartamento en Alquiler en Ur. El Señorial,"Ur. El Señorial, Cayma, Arequipa",3 dormitorios,USD1500
...,...,...,...,...
63,Apartamento en Alquiler en José Luis Bustamant...,"José Luis Bustamante y Rivero, Arequipa",2 dormitorios,1500
64,Apartamento en Alquiler en Miraflores,"Miraflores, Arequipa",3 dormitorios,USD1500
65,Apartamento en Alquiler en Vallecito,"Vallecito, Arequipa",3 dormitorios,3300
66,Apartamento en Alquiler en Cayma,"Cayma, Arequipa",2 dormitorios,1450


In [9]:
#Primero procederemos a extraer el tipo de cambio de la web y guardarlo como usd_to_pen para los valores en dólares. 
api_key = 'TU_CLAVE_DE_API'

url = f"https://open.er-api.com/v6/latest/USD?apikey={api_key}"

response = requests.get(url)
data = response.json()

if response.status_code == 200:
    usd_to_pen = data['rates']['PEN']
    print(f"El tipo de cambio de USD a PEN es: {usd_to_pen}")
else:
    print("No se pudo obtener el tipo de cambio.")

El tipo de cambio de USD a PEN es: 3.726831


In [10]:
# Función para convertir valores de USD a PEN
def convertir_a_pen_usd(valor, tipo_cambio):
    if valor.startswith('USD'):
        # Extraer el valor numérico (sin el prefijo 'USD')
        monto_usd = float(valor.replace('USD', ''))
        # Convertir a PEN
        monto_pen = monto_usd * tipo_cambio
        return monto_pen  # Devolver el valor convertido
    else:
        return valor  # Mantener el valor original si no tiene prefijo 'USD'

# Aplicar la función a la columna 'Precio' usando apply
data_properati['Precio'] = data_properati['Precio'].apply(lambda x: convertir_a_pen_usd(x, usd_to_pen))

# Convertir la columna 'Precio' a números (floats) para poder luego convertir la columna a enteros
data_properati['Precio'] = pd.to_numeric(data_properati['Precio'], errors='coerce')

# Redondear la columna 'Precio' a 0 decimales y convertir a enteros
data_properati['Precio'] = data_properati['Precio'].round(0).astype(int)

# Mostrar los primeros valores del DataFrame con los cambios
data_properati

,Detalle,Direccion,Dormitorios,Precio
0,Apartamento en Alquiler en Retamas,"Retamas, Yanahuara, Arequipa",3 dormitorios,2400
1,Apartamento en Alquiler en Sachaca,"Sachaca, Arequipa",3 dormitorios,1600
2,Apartamento en Alquiler en Vallecito,"Vallecito, Arequipa",3 dormitorios,2600
3,Apartamento en Alquiler en Los Alamos,"Los Alamos, Arequipa",3 dormitorios,5218
4,Apartamento en Alquiler en Ur. El Señorial,"Ur. El Señorial, Cayma, Arequipa",3 dormitorios,5590
...,...,...,...,...
63,Apartamento en Alquiler en José Luis Bustamant...,"José Luis Bustamante y Rivero, Arequipa",2 dormitorios,1500
64,Apartamento en Alquiler en Miraflores,"Miraflores, Arequipa",3 dormitorios,5590
65,Apartamento en Alquiler en Vallecito,"Vallecito, Arequipa",3 dormitorios,3300
66,Apartamento en Alquiler en Cayma,"Cayma, Arequipa",2 dormitorios,1450


In [11]:
#Encontramos que habian departamentos que se encontraban fuera de AQP, por lo que deberian ser eliminadas esas filas.
# Filtrar el DataFrame para mantener solo las filas que contienen 'Arequipa' en la columna 'Direccion' y eliminar el resto
data_properati = data_properati[~data_properati['Direccion'].str.contains('Mejia', case=False, na=False)]

#Visualizamos el dataframe ya limpio
data_properati

,Detalle,Direccion,Dormitorios,Precio
0,Apartamento en Alquiler en Retamas,"Retamas, Yanahuara, Arequipa",3 dormitorios,2400
1,Apartamento en Alquiler en Sachaca,"Sachaca, Arequipa",3 dormitorios,1600
2,Apartamento en Alquiler en Vallecito,"Vallecito, Arequipa",3 dormitorios,2600
3,Apartamento en Alquiler en Los Alamos,"Los Alamos, Arequipa",3 dormitorios,5218
4,Apartamento en Alquiler en Ur. El Señorial,"Ur. El Señorial, Cayma, Arequipa",3 dormitorios,5590
...,...,...,...,...
63,Apartamento en Alquiler en José Luis Bustamant...,"José Luis Bustamante y Rivero, Arequipa",2 dormitorios,1500
64,Apartamento en Alquiler en Miraflores,"Miraflores, Arequipa",3 dormitorios,5590
65,Apartamento en Alquiler en Vallecito,"Vallecito, Arequipa",3 dormitorios,3300
66,Apartamento en Alquiler en Cayma,"Cayma, Arequipa",2 dormitorios,1450


In [12]:
#Procederemos ahora a extraer las coordenadas de de cada direccion
# Utilizamos geolocator con un user agent cualquiera
geolocator = Nominatim(user_agent='Arequipa-Peru')

# Utilizamos geocode para extraer la location basada en el distrito
start = time.time()

# Geocode
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

# Establecemos la nueva columna de location
data_properati['Location'] = data_properati['Direccion'].apply(geocode)

# Una vez encontrada la location, extraemos las coordenadas en columnas separadas
data_properati['Latitud'] = data_properati['Location'].apply(lambda x: x.latitude if x else None)
data_properati['Longitud'] = data_properati['Location'].apply(lambda x: x.longitude if x else None)

# Finalizamos la función de tiempo
end = time.time()
elapsed = end - start

# Visualizamos el dataframe
print(str(elapsed) + " segundos")
#Visualizamos el dataframe
data_properati

68.21453547477722 segundos


C:\Users\Usuario\AppData\Local\Temp\ipykernel_13600\2947372035.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_properati['Location'] = data_properati['Direccion'].apply(geocode)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_13600\2947372035.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_properati['Latitud'] = data_properati['Location'].apply(lambda x: x.latitude if x else None)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_13600\2947372035.py:16: SettingWithCopyWarning: 
A value is tryi

,Detalle,Direccion,Dormitorios,Precio,Location,Latitud,Longitud
0,Apartamento en Alquiler en Retamas,"Retamas, Yanahuara, Arequipa",3 dormitorios,2400,None,NaN,NaN
1,Apartamento en Alquiler en Sachaca,"Sachaca, Arequipa",3 dormitorios,1600,"(Sachaca, Arequipa, 04013, Perú, (-16.42560499...",-16.425605,-71.575790
2,Apartamento en Alquiler en Vallecito,"Vallecito, Arequipa",3 dormitorios,2600,"(Vallecito, Chuquibamba, Condesuyos, Arequipa,...",-15.842004,-72.662038
3,Apartamento en Alquiler en Los Alamos,"Los Alamos, Arequipa",3 dormitorios,5218,"(Los Alamos, Guardia Civil, Paucarpata, Arequi...",-16.428007,-71.510951
4,Apartamento en Alquiler en Ur. El Señorial,"Ur. El Señorial, Cayma, Arequipa",3 dormitorios,5590,None,NaN,NaN
...,...,...,...,...,...,...,...
63,Apartamento en Alquiler en José Luis Bustamant...,"José Luis Bustamante y Rivero, Arequipa",2 dormitorios,1500,"(José Luis Bustamante y Rivero, Arequipa, Perú...",-16.433172,-71.523314
64,Apartamento en Alquiler en Miraflores,"Miraflores, Arequipa",3 dormitorios,5590,"(Miraflores, Arequipa, 04004, Perú, (-16.35018...",-16.350181,-71.462440
65,Apartamento en Alquiler en Vallecito,"Vallecito, Arequipa",3 dormitorios,3300,"(Vallecito, Chuquibamba, Condesuyos, Arequipa,...",-15.842004,-72.662038
66,Apartamento en Alquiler en Cayma,"Cayma, Arequipa",2 dormitorios,1450,"(Cayma, Arequipa, Perú, (-16.27939415, -71.460...",-16.279394,-71.460312


In [13]:
#Como observamos en el dataframe, no se pudieron obtener las locaciones de todas las direcciones, por ende tampoco su latitud y longitud.
#Podemos visualizar las direcciones erroneas. 
data_properati[data_properati['Location'].isnull()][['Direccion', 'Location']]

,Direccion,Location
0,"Retamas, Yanahuara, Arequipa",None
4,"Ur. El Señorial, Cayma, Arequipa",None
5,"Ur. Entel, Arequipa",None
9,"Chirisuya, Miraflores, Arequipa",None
14,"Ur. Entel, Arequipa",None
16,"Ur. Santa Maria, José Luis Bustamante y Rivero...",None
21,"Ur. Santa Cecilia, Yanahuara, Arequipa",None
29,"Retamas, Yanahuara, Arequipa",None
31,"Ur. El Señorial, Cayma, Arequipa",None
32,"Retamas, Yanahuara, Arequipa",None


In [14]:
# Dividimos cada dirección por las comas y conservamos todo después de la primera coma 

# funcion para definir que cuando location encuentre  valores nulos, se dirija a la columna 'Direccion' y reemplace los valores
def replace_null_with_district(row):
    location = row['Location']
    direccion = row['Direccion']
    if pd.isnull(location):
        new_direccion = direccion.split(',')[1].strip() if (isinstance(direccion, str) and ',' in direccion) else direccion
        #agregaremos ", Arequipa" a todas las direcciones que sean modificadas.
        return f"{new_direccion}, Arequipa" if isinstance(new_direccion, str) else new_direccion
    else:
        return direccion

data_properati['Direccion'] = data_properati.apply(replace_null_with_district, axis=1, result_type='reduce')

#Visualizamos el Dataframe con la columna ya modificada
data_properati[data_properati['Latitud'].isnull()][['Direccion', 'Location']]

C:\Users\Usuario\AppData\Local\Temp\ipykernel_13600\4235584878.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_properati['Direccion'] = data_properati.apply(replace_null_with_district, axis=1, result_type='reduce')


,Direccion,Location
0,"Yanahuara, Arequipa",None
4,"Cayma, Arequipa",None
5,"Arequipa, Arequipa",None
9,"Miraflores, Arequipa",None
14,"Arequipa, Arequipa",None
16,"José Luis Bustamante y Rivero, Arequipa",None
21,"Yanahuara, Arequipa",None
29,"Yanahuara, Arequipa",None
31,"Cayma, Arequipa",None
32,"Yanahuara, Arequipa",None


In [15]:
#Procederemos nuevamente a extraer las coordenadas de de cada direccion ya con las direcciones modificadas
# Utilizamos geolocator con un user agent cualquiera
geolocator = Nominatim(user_agent='Arequipa-Peru')

# Utilizamos geocode para extraer la location basada en el distrito
start = time.time()

# Geocode
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

# Establecemos la nueva columna de location
data_properati['Location'] = data_properati['Direccion'].apply(geocode)

# Una vez encontrada la location, extraemos las coordenadas en columnas separadas
data_properati['Latitud'] = data_properati['Location'].apply(lambda x: x.latitude if x else None)
data_properati['Longitud'] = data_properati['Location'].apply(lambda x: x.longitude if x else None)

# Finalizamos la función de tiempo
end = time.time()
elapsed = end - start

# Visualizamos el dataframe
print(str(elapsed) + " segundos")
#Visualizamos el dataframe a ver si existen valores nulos.
data_properati[data_properati['Latitud'].isnull()][['Direccion', 'Latitud']]

67.54540348052979 segundos


C:\Users\Usuario\AppData\Local\Temp\ipykernel_13600\2293357693.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_properati['Location'] = data_properati['Direccion'].apply(geocode)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_13600\2293357693.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_properati['Latitud'] = data_properati['Location'].apply(lambda x: x.latitude if x else None)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_13600\2293357693.py:16: SettingWithCopyWarning: 
A value is tryi

,Direccion,Latitud


In [16]:
# Concatenar los DataFrames uno encima del otro
data_alquiler = pd.concat([data_adondevivir, data_properati])

#Vamos a añadir una columna con el tipo de alojamiento
tipo_alojamiento = 'Alquiler mensual'

data_alquiler['Tipo']= tipo_alojamiento

#Eliminamos los espacios innecesarios en el dataframe
data_alquiler = data_alquiler.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

# Mostrar el DataFrame resultante
print(data_alquiler.dtypes)
data_alquiler

Detalle         object
Direccion       object
Dormitorios     object
Precio           int32
Descripcion     object
Location       float64
Latitud        float64
Longitud       float64
Tipo            object
dtype: object


,Detalle,Direccion,Dormitorios,Precio,Descripcion,Location,Latitud,Longitud,Tipo
0,Alquilo Depa Amoblado en Vallecito,"Vallecito, Arequipa",3 dorm.,2600,Moderno y amplio departamento totalmente amobl...,NaN,-15.842004,-72.662038,Alquiler mensual
1,Alquilo Exelente Departamento Amoblado Cerca a...,"Arequipa, Arequipa",3 dorm.,3500,Alquilo moderno Departamento amoblado. Cuenta ...,NaN,-16.398867,-71.536961,Alquiler mensual
2,Alquilo Moderno y Amplio Departamento en Cerca...,"Arequipa, Arequipa",3 dorm.,1600,Alquilo moderno y amplio Departamento en Cerca...,NaN,-16.398867,-71.536961,Alquiler mensual
3,Hermoso Departamento Amoblado en Umacollo,"Yanahuara, Arequipa",3 dorm.,1700,Ubicado a 06 cuadras de la universidad catolic...,NaN,-16.388182,-71.542054,Alquiler mensual
4,Alquilo Comodo Departamento Muy Bien Ubicado e...,"Cayma, Arequipa",3 dorm.,1500,Alquilo cómodo Departamento sin amoblar en Urb...,NaN,-16.279394,-71.460312,Alquiler mensual
...,...,...,...,...,...,...,...,...,...
63,Apartamento en Alquiler en José Luis Bustamant...,"José Luis Bustamante y Rivero, Arequipa",2 dormitorios,1500,NaN,NaN,-16.433172,-71.523314,Alquiler mensual
64,Apartamento en Alquiler en Miraflores,"Miraflores, Arequipa",3 dormitorios,5590,NaN,NaN,-16.350181,-71.462440,Alquiler mensual
65,Apartamento en Alquiler en Vallecito,"Vallecito, Arequipa",3 dormitorios,3300,NaN,NaN,-15.842004,-72.662038,Alquiler mensual
66,Apartamento en Alquiler en Cayma,"Cayma, Arequipa",2 dormitorios,1450,NaN,NaN,-16.279394,-71.460312,Alquiler mensual


In [51]:
#Vamos a extraer el distrito de cada direccion.
# Crear la columna 'Distrito' usando apply y la función buscar_distrito. La funcion ya fue definida.
data_alquiler['Distrito'] = data_alquiler['Direccion'].apply(lambda x: buscar_distrito(x))
#Visualizamos en el dataframe las direcciones que no cuentan con distrito
data_alquiler[data_alquiler['Distrito'].isnull()][['Distrito', 'Direccion']]

,Distrito,Direccion
0,None,"Vallecito, Arequipa"
1,None,"Arequipa, Arequipa"
2,None,"Arequipa, Arequipa"
14,None,"Arequipa, Arequipa"
17,None,"Arequipa, Arequipa"
19,None,"Arequipa, Arequipa"
21,None,"Selva Alegre, Arequipa"
38,None,"Uchumayo, Arequipa"
40,None,"Jose Luis Bustamante Y Rivero, Arequipa"
41,None,"La Negrita, Arequipa"


In [52]:
# Vamos a Reemplazar las columnas de manera manual
# Diccionario de mapeo de direcciones a distritos
mapeo_manual = {
    "Vallecito": "Arequipa",
    "Selva Alegre": "Alto Selva Alegre",
    "Jose Luis Bustamante Y Rivero": "José Luis Bustamante y Rivero",
    "Uchumayo": "Uchumayo",
    "Umacollo": "Yanahuara"
}

# Función para asignar el distrito basado en la dirección, solo si Distrito es NaN
def asignar_distrito(row):
    if pd.isnull(row['Distrito']):
        direccion = row['Direccion']
        for key in mapeo_manual.keys():
            if key in direccion:
                return mapeo_manual[key]
    return row['Distrito']  # Mantener el valor existente si no se encuentra ninguna coincidencia

# Aplicar la función a las filas donde 'Distrito' esté vacío
data_alquiler['Distrito'] = data_alquiler.apply(asignar_distrito, axis=1)
#Visualizamos en el dataframe las direcciones que no cuentan con distrito
data_alquiler[data_alquiler['Distrito'].isnull()][['Distrito', 'Direccion']]

,Distrito,Direccion
1,None,"Arequipa, Arequipa"
2,None,"Arequipa, Arequipa"
14,None,"Arequipa, Arequipa"
17,None,"Arequipa, Arequipa"
19,None,"Arequipa, Arequipa"
41,None,"La Negrita, Arequipa"
44,None,"Arequipa, Arequipa"
50,None,"Cercado, Arequipa"
58,None,"Arequipa, Arequipa"
68,None,"Arequipa, Arequipa"


In [53]:
#El resto de direcciones corresponden al centro de la ciudad, es decir al distrito de Arequipa.
# Reemplazamos los valores en blanco en la columna 'Distrito' por 'Arequipa'
data_alquiler['Distrito'].fillna('Arequipa', inplace=True)
# Añadir ", Arequipa" a todos los valores de la columna 'Distrito'
data_alquiler['Distrito'] = data_alquiler['Distrito'].apply(lambda x: x + ', Arequipa' if pd.notnull(x) else x)
#Visualizamos el dataframe
data_alquiler[['Distrito', 'Direccion']]

,Distrito,Direccion
0,"Arequipa, Arequipa","Vallecito, Arequipa"
1,"Arequipa, Arequipa","Arequipa, Arequipa"
2,"Arequipa, Arequipa","Arequipa, Arequipa"
3,"Yanahuara, Arequipa","Yanahuara, Arequipa"
4,"Cayma, Arequipa","Cayma, Arequipa"
...,...,...
63,"José Luis Bustamante y Rivero, Arequipa","José Luis Bustamante y Rivero, Arequipa"
64,"Miraflores, Arequipa","Miraflores, Arequipa"
65,"Arequipa, Arequipa","Vallecito, Arequipa"
66,"Cayma, Arequipa","Cayma, Arequipa"


In [54]:
#Procedemos a guardar el dataframe en un archivo xlsx nuevo
nombre_archivo = 'Data Apartamentos Alquiler.xlsx'

data_alquiler.to_excel(nombre_archivo, index=False)  # El argumento index=False evita que se incluyan los índices en el archivo

print(f'DataFrame guardado en {nombre_archivo}')

DataFrame guardado en Data Apartamentos Alquiler.xlsx


In [55]:
# Concatenar los DataFrames uno encima del otro de la data de Airbn y Alquiler Mensual
data_apartamentos = pd.concat([data_airbnb, data_alquiler])

#Eliminamos los espacios innecesarios en el dataframe
data_apartamentos = data_apartamentos.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

# Mostrar el DataFrame resultante
print(data_apartamentos.dtypes)
data_apartamentos

Detalle         object
Direccion       object
Dormitorios     object
Camas           object
Precio           int32
Personas        object
Descripcion     object
Distrito        object
Tipo            object
Location       float64
Latitud        float64
Longitud       float64
dtype: object


,Detalle,Direccion,Dormitorios,Camas,Precio,Personas,Descripcion,Distrito,Tipo,Location,Latitud,Longitud
0,"Hermoso departamento en el centro ciudad, casa...","420 Melgar, Arequipa, Perú",Apartamento de 2 dormitorios,1 cama individual,134,2 adultos · 0 niños · 1 habitación,"El Hermoso departamento en el centro ciudad, c...","Arequipa, Arequipa",Airbnb,NaN,-16.398867,-71.536961
1,Departamento amplio y bonito en Arequipa 2do Piso,"Urbanización Colegio de ingenieros A9, Cerro C...",Apartamento de 1 dormitorio,1 cama doble,159,2 adultos · 0 niños · 1 habitación,El Departamento Amoblado en Urbanización Coleg...,"Cerro Colorado, Arequipa",Airbnb,NaN,-16.297999,-71.573948
2,Big apartment near historical center,"506 Lima, Arequipa, Perú",Apartamento,1 cama doble grande,179,2 adultos · 0 niños · 1 habitación,El Big apartment near historical center está s...,"Arequipa, Arequipa",Airbnb,NaN,-16.398867,-71.536961
3,Furnished Aparments Arequipa,"Urbanización Campiña Dorada C - 1, Arequipa, Perú",Apartamento Estándar,2 camas dobles,115,2 adultos · 0 niños · 1 habitación,El Furnished Apartments Arequipa está situado ...,"Arequipa, Arequipa",Airbnb,NaN,-16.398867,-71.536961
4,Hermoso departamento en exclusivo Condominio,"LOS PARQUES CASA CLUB CALLE HERNANDO DE LUQUE,...",Apartamento de 1 dormitorio,1 cama doble extragrande,109,2 adultos · 0 niños · 1 habitación,El Hermoso departamento en exclusivo Condomini...,"Arequipa, Arequipa",Airbnb,NaN,-16.398867,-71.536961
...,...,...,...,...,...,...,...,...,...,...,...,...
63,Apartamento en Alquiler en José Luis Bustamant...,"José Luis Bustamante y Rivero, Arequipa",2 dormitorios,NaN,1500,NaN,NaN,"José Luis Bustamante y Rivero, Arequipa",Alquiler mensual,NaN,-16.433172,-71.523314
64,Apartamento en Alquiler en Miraflores,"Miraflores, Arequipa",3 dormitorios,NaN,5590,NaN,NaN,"Miraflores, Arequipa",Alquiler mensual,NaN,-16.350181,-71.462440
65,Apartamento en Alquiler en Vallecito,"Vallecito, Arequipa",3 dormitorios,NaN,3300,NaN,NaN,"Arequipa, Arequipa",Alquiler mensual,NaN,-15.842004,-72.662038
66,Apartamento en Alquiler en Cayma,"Cayma, Arequipa",2 dormitorios,NaN,1450,NaN,NaN,"Cayma, Arequipa",Alquiler mensual,NaN,-16.279394,-71.460312


In [56]:
#Procedemos a guardar el dataframe en un archivo xlsx nuevo
nombre_archivo = 'Data Apartamentos Limpio.xlsx'

data_apartamentos.to_excel(nombre_archivo, index=False)  # El argumento index=False evita que se incluyan los índices en el archivo

print(f'DataFrame guardado en {nombre_archivo}')

DataFrame guardado en Data Apartamentos Limpio.xlsx
